In [7]:
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from DS_healpix_fragmentation import radec2pix
from DS_Planck_Unet import *

In [5]:
df = None
with fits.open('/home/rt2122/Data/DR5_cluster-catalog_v1.0b3.fits') as hdul:
    tbl = Table(hdul[1].data)
    names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
    df = tbl[names].to_pandas()


In [7]:
df.head()

,name,RADeg,decDeg,SNR,y_c,err_y_c,fixed_SNR,fixed_y_c,fixed_err_y_c,template,...,RMDESY3_LAMBDA_CHISQ_E,CAMIRA,CAMIRA_N_mem,opt_RADeg,opt_decDeg,opt_positionSource,notes,knownLens,knownLensRefCode,warnings
0,ACT-CL J0000.0-3837,0.020896,-38.626729,4.801052,0.326241,0.067952,4.638507,0.461301,0.099450,Arnaud_M4e14_z0p2,...,2.881614,False,-99.0,0.023776,-38.626737,RMDESY3,,,,
1,ACT-CL J0000.2-4357,0.061318,-43.955645,7.676262,0.593745,0.077348,7.581173,0.725846,0.095743,Arnaud_M1e14_z0p2,...,4.824760,False,-99.0,0.071517,-43.957593,Vis-BCG,,,,
2,ACT-CL J0000.3-2805,0.089905,-28.091255,5.655515,0.675573,0.119454,4.649276,0.885057,0.190364,Arnaud_M8e14_z0p2,...,-99.000000,False,-99.0,0.096956,-28.103488,Vis-BCG,,,,
3,ACT-CL J0000.4-0102,0.113536,-1.044805,4.968499,0.293465,0.059065,4.426424,0.370215,0.083637,Arnaud_M8e14_z0p2,...,3.726360,False,-99.0,0.116001,-1.043040,Vis-BCG,,,,
4,ACT-CL J0000.4-0552,0.113978,-5.880626,4.583042,1.067623,0.232951,4.309756,0.466331,0.108203,Arnaud_M1e14_z1p2,...,-99.000000,False,-99.0,0.110460,-5.884459,Vis-BCG,,,,


In [8]:
df.shape

(4195, 51)

In [9]:
planck_z = pd.read_csv('/home/rt2122/Data/clusters/planck_z.csv')

In [12]:
act_sc = SkyCoord(ra=df['RADeg']*u.degree, dec=df['decDeg']*u.degree, frame='icrs')
p_sc = SkyCoord(ra=planck_z['RA']*u.degree, dec=planck_z['DEC']*u.degree, frame='icrs')

In [13]:
idx, sep2d, _ = act_sc.match_to_catalog_sky(p_sc)

In [16]:
np.count_nonzero(sep2d.degree < 5 / 60)

346

In [17]:
df = df[sep2d.degree < 5 / 60]
df.index = np.arange(df.shape[0])

In [18]:
df.shape

(346, 51)

In [19]:
planck_no_z = pd.read_csv('/home/rt2122/Data/clusters/planck_no_z.csv')

In [21]:
pnz_sc = SkyCoord(
    ra=planck_no_z['RA']*u.degree, dec=planck_no_z['DEC']*u.degree, frame='icrs')
act_sc = SkyCoord(ra=df['RADeg']*u.degree, dec=df['decDeg']*u.degree, frame='icrs')

In [22]:
idx, sep2d, _ = act_sc.match_to_catalog_sky(pnz_sc)

In [24]:
np.count_nonzero(sep2d.degree < 15 / 60)

0

In [25]:
mcxcwp = pd.read_csv('/home/rt2122/Data/clusters/mcxcwp.csv')
m_sc = SkyCoord(
    ra=mcxcwp['RA']*u.degree, dec=mcxcwp['DEC']*u.degree, frame='icrs')
act_sc = SkyCoord(ra=df['RADeg']*u.degree, dec=df['decDeg']*u.degree, frame='icrs')

In [26]:
idx, sep2d, _ = act_sc.match_to_catalog_sky(m_sc)

In [29]:
np.count_nonzero(sep2d.degree < 5 / 60)

0

In [30]:
df = df.rename({'RADeg' : 'RA', 'decDeg' : 'DEC'}, axis='columns')

In [31]:
df.head()

,name,RA,DEC,SNR,y_c,err_y_c,fixed_SNR,fixed_y_c,fixed_err_y_c,template,...,RMDESY3_LAMBDA_CHISQ_E,CAMIRA,CAMIRA_N_mem,opt_RADeg,opt_decDeg,opt_positionSource,notes,knownLens,knownLensRefCode,warnings
0,ACT-CL J0001.9+1203,0.482636,12.061679,7.293848,0.723885,0.099246,6.446910,1.014815,0.157411,Arnaud_M8e14_z0p2,...,-99.000000,False,-99.000,0.493217,12.066475,Vis-BCG,,,,
1,ACT-CL J0003.2-0605,0.802677,-6.089536,20.129664,1.669388,0.082932,19.568297,2.148085,0.109774,Arnaud_M2e14_z0p2,...,-99.000000,False,-99.000,0.798265,-6.091704,Vis-BCG,,,,
2,ACT-CL J0003.8+0203,0.954186,2.062602,5.478078,0.396478,0.072375,5.420873,0.494142,0.091155,Arnaud_M8e14_z0p4,...,3.419855,True,31.927,0.956512,2.067147,Vis-BCG,,,,
3,ACT-CL J0005.9-3443,1.495625,-34.724928,5.392566,0.367563,0.068161,5.195207,0.518366,0.099778,Arnaud_M8e14_z0p2,...,4.453872,False,-99.000,1.498445,-34.721933,Vis-BCG,,,,
4,ACT-CL J0006.3+1051,1.584596,10.857148,7.058319,0.730762,0.103532,6.595807,1.100267,0.166813,Arnaud_M8e14_z0p2,...,-99.000000,False,-99.000,1.584371,10.864830,Vis-BCG,,,,Redshift > 0.6 prior applied in photo-z estimate


In [32]:
df.to_csv('/home/rt2122/Data/clusters/act.csv', index=False)

In [2]:
act = pd.read_csv('/home/rt2122/Data/clusters/act.csv')

In [3]:
act['pix2'] = np.zeros(len(act), dtype=int)

In [5]:
act['pix2'] = radec2pix(act['RA'], act['DEC'], nside=2)

In [8]:
for i in test_pix:
    print(i, np.count_nonzero(act['pix2'] == i))

6 0


In [9]:
for i in val_pix:
    print(i, np.count_nonzero(act['pix2'] == i))

9 10
38 14
41 34


In [11]:
for i in train_pix:
    if np.count_nonzero(act['pix2'] == i):
        print(i, np.count_nonzero(act['pix2'] == i))

0 1
2 15
8 13
10 6
11 10
13 1
14 2
15 23
16 5
20 2
24 3
28 8
32 25
33 26
34 17
35 6
36 22
37 10
40 22
42 11
43 13
44 23
45 16
46 8
